<a href="https://colab.research.google.com/github/Ahtesham519/Genrative_Deep_learning_v2_2023/blob/main/Energy_Based_Models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
%load_ext autoreload
%autoreload 2

import numpy as np

import tensorflow as tf
from tensorflow.keras import (
    datasets,
    layers,
    models,
    optimizers,
    activations,
    metrics,
    callbacks,

)
import random

#0.  Parameters

In [13]:
IMAGE_SIZE = 32
CHANNELS = 1
STEP_SIZE = 10
STEPS = 60
NOISE = 0.005
ALPHA = 0.1
GRADIENT_CLIP = 0.03
BATCH_SIZE = 128
BUFFER_SIZE = 8192
LEARNING_RATE = 0.0001
EPOCHS = 60
LOAD_MODEL = False


In [14]:
#Load the data
(x_train, _), (x_test, _) = datasets.mnist.load_data()

In [15]:
#Preprocess the data

def preprocess(imgs):
  """
  Normalize and reshape the images
  """
  imgs = (imgs.astype("float32") - 127.5)/ 127.5
  imgs = np.pad(imgs, ((0,0) ,( 2,2) , (2,2)) , constant_values = -1.0)
  imgs = np.expand_dims(imgs, -1)
  return imgs

  x_train = preprocess(x_train)
  x_test = preprocess(x_test)



In [16]:
x_train = tf.data.Dataset.from_tensor_slices(x_train).batch(BATCH_SIZE)
x_test = tf.data.Dataset.from_tensor_slices(x_test).batch(BATCH_SIZE)

In [17]:
#Show some items of the clothing from the training set
x_train

<_BatchDataset element_spec=TensorSpec(shape=(None, 28, 28), dtype=tf.uint8, name=None)>

#2. Build the EBM network

In [18]:
ebm_input = layers.Input(shape=(IMAGE_SIZE, IMAGE_SIZE , CHANNELS))
x = layers.Conv2D(
    16,kernel_size = 5, strides=2, padding= "same", activation= activations.swish
)(ebm_input)
x = layers.Conv2D(
    32, kernel_size = 3, strides= 2 , padding = "same", activation = activations.swish
)(x)
x = layers.Conv2D(
    64, kernel_size = 3, strides=2, padding = "same" , activation = activations.swish
)(x)
x = layers.Conv2D(
    64, kernel_size = 3, strides = 2, padding = "same", activation = activations.swish
)(x)
x = layers.Flatten()(x)
x = layers.Dense(64, activation = activations.swish)(x)
ebm_output = layers.Dense(1)(x)
model = models.Model(ebm_input,ebm_output)
model.summary()


Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 32, 32, 1)]       0         
                                                                 
 conv2d_4 (Conv2D)           (None, 16, 16, 16)        416       
                                                                 
 conv2d_5 (Conv2D)           (None, 8, 8, 32)          4640      
                                                                 
 conv2d_6 (Conv2D)           (None, 4, 4, 64)          18496     
                                                                 
 conv2d_7 (Conv2D)           (None, 2, 2, 64)          36928     
                                                                 
 flatten_1 (Flatten)         (None, 256)               0         
                                                                 
 dense_2 (Dense)             (None, 64)                1644

In [19]:
if LOAD_MODEL:
  model.load_weights("./models/model.h5")

#2. Set up a Langevin sampler function

In [20]:
#Function to generate samples using Langevin Dynamics
def generate_samples(
    model, inps_imgs, steps, step_size, noise, return_img_per_step = False
):
    imgs_per_step = []
    for _ in range(steps):
      inp_imgs += tf.random.normal(inp_imgs.shape, mean = 0 , stddev = noise)
      inp_imgs = tf.clip_by_value(inp_imgs, -1.0 , 1.0)
      with tf.GradientTape() as tape:
        tape.watch(inp_imgs)
        out_score = model(inp_imgs)
      grads = tape.gradient(out_score, inp_imgs)
      grads = tf.clip_by_value(grads, -GRADIENT_CLIP , GRADIENT_CLIP)
      inp_imgs += step_size * grads
      inp_imgs = tf.clip_by_value(inp_imgs, -1.0 , 1.0)
      if return_img_per_step:
        imgs_per_step.append(inp_imgs)
    if return_img_per_step:
      return tf.stack(imgs_per_step, axis = 0)
    else:
      return inp_imgs

#3. Set up a buffer to store examples

In [21]:
class Buffer:
  def __init__(self, model):
    super().__init__()
    self.model = model
    self.examples = [
        tf.random.uniform(shape =(1, IMAGE_SIZE , IMAGE_SIZE , CHANNELS)) * 2
        -1
        for _ in range(BATCH_SIZE)
    ]

  def sample_new_exmps(self, steps,step_size , noise):
    n_new = np.random.bionomial(BATCH_SIZE , 0.05)
    rand_imgs = (
        tf.random.uniform((n_new , IMAGE_SIZE, IMAGE_SIZE, CHANNELS))* 2-1
    )
    old_imgs = tf.concat(
          random.choices(self.examples, k = BATCH_SIZE - n_new), axis = 0
    )
    inp_imgs = tf.concat([rand_imgs, old_imgs] , axis = 0)
    inp_imgs = generate_samples(
        self.model, inp_imgs, steps = steps , step_size = step_size, noise = noise
    )
    self.examples = tf.split(inp_imgs, BATCH_SIZE, axis = 0) + self.examples
    self.examples = self.examples[:BUFFER_SIZE]
    return inp_imgs

In [23]:
class EBM(models.Model):
  def __init__ (self):
    super(EBM, self).__init__()
    self.model = model
    self.buffer = Buffer(self.model)
    self.alpha = ALPHA
    self.loss_metric = metrics.Mean(name = "loss")
    self.reg_loss_metric = metrics.Mean(name= "reg")
    self.cdiv_loss_metric = metrics.Mean(name= 'cdiv')
    self.real_out_metric = metrics.Mean(name = "real")
    self.fake_out_metric = metrics.Mean(name = "fake")

  @property
  def metrics(self):
    return [
        self.loss_metric,
        self.reg_loss_metric,
        self.cdiv_loss_metric,
        self.real_out_metric,
        self.fake_out_metric,

    ]

  def train_step(self, real_imgs):
    real_imgs += tf.random.normal(
        shape= tf.shape(real_imgs), mean = 0, stddev = NOISE
    )
    real_imgs = tf.clip_by_value(real_imgs, -1.0, 1.0)
    fake_imgs = self.buffer.sample_new_exmps(
        steps = STEPS, step_size = STEP_SIZE , noise = NOISE
    )
    inp_imgs = tf.concat([real_imgs, fake_imgs], axis = 0)
    with tf.GradientTape() as training_tape:
      real_out , fake_out = tf.split(self.model(inp_imgs) , 2, axis = 0)
      cdiv_loss = tf.reduce_mean(fake_out, axis = 0) - tf.reduce_mean(
          real_out, axis = 0
      )
      reg_loss = self.alpha * tf.reduce_mean(
          real_out **2 + fake_out **2 , axis = 0
      )
      loss = cdiv_loss +reg_loss
      grads = training_tape.gradient(loss, self.model.trainable_variables)
      self.optimizer.apply_gradients(
          zip(grads, self.model.trainable_variables)
      )
      self.loss_metric.update_state(loss)
      self.reg_loss_metric.update_state(reg_loss)
      self.cdiv_loss_metric.update_state(cdiv_loss)
      self.real_out_metric.update_state(tf.reduce_mean(real_out, axis =0))
      self.fake_out_metric.update_state(tf.reduce_mean(fake_out, axis = 0))
      return {m.name: m.result() for m in self.metrics}

    def test_step(self, real_imgs):
      batch_size = real_imgs.shape[0]
      fake_imgs = (
          tf.random.uniform((batch_size, IMAGE_SIZE , IMAGE_SIZE , CHANNELS))
          * 2
          - 1
      )
      inp_imgs = tf.concat([real_imgs, fake_imgs] , axis = 0)
      real_out, fake_out = tf.split(self.model(inp_imgs) , 2, axis = 0)
      cdiv = tf.reduce_mean(fake_out, axis = 0) - tf.reduce_mean(
          real_out , axis = 0
      )
      self.cdiv_loss_metric.update_state(cdiv)
      self.real_out_metric.update_state(tf.reduce_mean(real_out, axis = 0))
      self.fake_out_metric.update_state(tf.reduce_mean(fake_out, axis = 0))
      return {m.name : m.result() for m in self.metrics[2:]}











In [24]:
ebm = EBM()

#3. Train the EBM network